---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [169]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
# from sklearn.model_selection import 

def blight_model():
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve, auc, roc_auc_score
    # Your code here
    test = pd.read_csv('test.csv')
    train = pd.read_csv('train.csv',encoding = 'ISO-8859-1' )
#     address = pd.read_csv('addresses.csv')
#     latlons = pd.read_csv('latlons.csv')
    train = train.dropna(subset=['compliance'])
    
    ucats_an = set(train['agency_name'])|{'<unknown>'}
    ucats_co = set(train['country'])|{'<unknown>'}
    ucats_st = set(train['state'])|{'<unknown>'}
    ucats_ct = set(train['city'])|{'<unknown>'}
    ucats_dp = set(train['disposition'])|{'<unknown>'}
    ucats_dc = set(train['violation_description'])|{'<unknown>'}
    ucats_vc = set(train['violation_code'])|{'<unknown>'}

    train['an']= pd.Categorical(train['agency_name'],categories=ucats_an,ordered=False).fillna('<unknown>').codes
    test['an']= pd.Categorical(test['agency_name'],categories=ucats_an,ordered=False).fillna('<unknown>').codes
    train['co']= pd.Categorical(train['country'],categories=ucats_co,ordered=False).fillna('<unknown>').codes
    test['co']= pd.Categorical(test['country'],categories=ucats_co,ordered=False).fillna('<unknown>').codes
    train['st']= pd.Categorical(train['state'],categories=ucats_st,ordered=False).fillna('<unknown>').codes
    test['st']= pd.Categorical(test['state'],categories=ucats_st,ordered=False).fillna('<unknown>').codes
    train['ct']= pd.Categorical(train['city'],categories=ucats_ct,ordered=False).fillna('<unknown>').codes
    test['ct']= pd.Categorical(test['city'],categories=ucats_ct,ordered=False).fillna('<unknown>').codes
    train['dp']= pd.Categorical(train['disposition'],categories=ucats_dp,ordered=False).fillna('<unknown>').codes
    test['dp']= pd.Categorical(test['disposition'],categories=ucats_dp,ordered=False).fillna('<unknown>').codes
    train['dc']= pd.Categorical(train['violation_description'],categories=ucats_dc,ordered=False).fillna('<unknown>').codes
    test['dc']= pd.Categorical(test['violation_description'],categories=ucats_dc,ordered=False).fillna('<unknown>').codes
    train['vc']= pd.Categorical(train['violation_code'],categories=ucats_vc,ordered=False).fillna('<unknown>').codes
    test['vc']= pd.Categorical(test['violation_code'],categories=ucats_vc,ordered=False).fillna('<unknown>').codes

#     temp = (pd.to_datetime(train.hearing_date)-pd.to_datetime(train.ticket_issued_date)).astype('timedelta64[M]')
#     ucats_td = set(filter(lambda x: x==x, temp))|{'<unknown>'}
#     train['td'] = pd.Categorical(temp, categories=ucats_td, ordered=True).fillna('<unknown>').codes

#     temp = (pd.to_datetime(test.hearing_date)-pd.to_datetime(test.ticket_issued_date)).astype('timedelta64[M]')
#     test['td'] = pd.Categorical(temp, categories=ucats_td, ordered=True).fillna('<unknown>').codes
    
    features = ['an','co','st','dp','vc','fine_amount','late_fee','discount_amount','judgment_amount']

    X = train[features]
    y = train['compliance']
    X_test = test[features]
    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X, y)
    y_test_scores = clf.predict_proba(X_test)[:,1]
    compliance = pd.Series(y_test_scores, index=test['ticket_id'])
    compliance.astype(np.float32)
    return compliance# Your answer here
# blight_model()
# ans[0:5]

In [170]:
# test = pd.read_csv('test.csv')
# train = pd.read_csv('train.csv',encoding = 'ISO-8859-1' )
# print('train shape is: {}'.format(train.shape))
# print('test shape is: {}'.format(test.shape))
# train = train.dropna(subset=['compliance'])
# print('train shape is: {}'.format(train.shape))

In [171]:
# print(train.columns)
# print(test.columns)

In [172]:
# print('agency name set: {}'.format(len(set(train.agency_name))))
# print(set(train.agency_name))
# print('****************************************************************************************************')
# print('state: {}'.format(len(set(train.state))))
# print(set(train.state))
# print('****************************************************************************************************')
# print('country: {}'.format(len(set(train.country))))
# print(set(train.country))
# print('****************************************************************************************************')
# print('disposition: {}'.format(len(set(train.disposition))))
# print(set(train.disposition))
# print('****************************************************************************************************')
# print('violation_code: {}'.format(len(set(train.violation_code))))
# print('city: {}'.format(len(set(train.city))))
# print('fine_amount {}'.format(len(set(train.fine_amount))))
# print('admin_fee {}'.format(len(set(train.admin_fee))))
# print('state_fee {}'.format(len(set(train.state_fee))))
# print('late_fee {}'.format(len(set(train.late_fee))))
# print('discount_amount {}'.format(len(set(train.discount_amount))))
# print('judgment_amount {}'.format(len(set(train.judgment_amount))))
# print('clean_up_cost {}'.format(len(set(train.clean_up_cost))))
# print('violation_description {}'.format(len(set(train.violation_description))))
# print('****************************************************************************************************')

In [173]:
# ucats_an = set(train['agency_name'])|{'<unknown>'}
# ucats_co = set(train['country'])|{'<unknown>'}
# ucats_st = set(train['state'])|{'<unknown>'}
# ucats_ct = set(train['city'])|{'<unknown>'}
# ucats_dp = set(train['disposition'])|{'<unknown>'}
# ucats_dc = set(train['violation_description'])|{'<unknown>'}
# ucats_vc = set(train['violation_code'])|{'<unknown>'}

# train['an']= pd.Categorical(train['agency_name'],categories=ucats_an,ordered=False).fillna('<unknown>').codes
# test['an']= pd.Categorical(test['agency_name'],categories=ucats_an,ordered=False).fillna('<unknown>').codes
# train['co']= pd.Categorical(train['country'],categories=ucats_co,ordered=False).fillna('<unknown>').codes
# test['co']= pd.Categorical(test['country'],categories=ucats_co,ordered=False).fillna('<unknown>').codes
# train['st']= pd.Categorical(train['state'],categories=ucats_st,ordered=False).fillna('<unknown>').codes
# test['st']= pd.Categorical(test['state'],categories=ucats_st,ordered=False).fillna('<unknown>').codes
# train['ct']= pd.Categorical(train['city'],categories=ucats_ct,ordered=False).fillna('<unknown>').codes
# test['ct']= pd.Categorical(test['city'],categories=ucats_ct,ordered=False).fillna('<unknown>').codes
# train['dp']= pd.Categorical(train['disposition'],categories=ucats_dp,ordered=False).fillna('<unknown>').codes
# test['dp']= pd.Categorical(test['disposition'],categories=ucats_dp,ordered=False).fillna('<unknown>').codes
# train['dc']= pd.Categorical(train['violation_description'],categories=ucats_dc,ordered=False).fillna('<unknown>').codes
# test['dc']= pd.Categorical(test['violation_description'],categories=ucats_dc,ordered=False).fillna('<unknown>').codes
# train['vc']= pd.Categorical(train['violation_code'],categories=ucats_vc,ordered=False).fillna('<unknown>').codes
# test['vc']= pd.Categorical(test['violation_code'],categories=ucats_vc,ordered=False).fillna('<unknown>').codes

# temp = (pd.to_datetime(train.hearing_date)-pd.to_datetime(train.ticket_issued_date)).astype('timedelta64[M]')
# ucats_td = set(filter(lambda x: x==x, temp))|{'<unknown>'}
# train['td'] = pd.Categorical(temp, categories=ucats_td, ordered=True).fillna('<unknown>').codes

# temp = (pd.to_datetime(test.hearing_date)-pd.to_datetime(test.ticket_issued_date)).astype('timedelta64[M]')
# test['td'] = pd.Categorical(temp, categories=ucats_td, ordered=True).fillna('<unknown>').codes

In [174]:
# features = ['an','co','st','dp','vc','fine_amount','late_fee','discount_amount','judgment_amount']
# X = train[features]
# y = train['compliance']
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

# X_train.shape

In [175]:
# X_train.head()

In [176]:
# clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train, y_train)
# print(clf.score(X_train, y_train))
# print(clf.score(X_test, y_test))


In [177]:
# y_train_scores = clf.predict_proba(X_train)[:,1]
# y_test_scores = clf.predict_proba(X_test)[:,1]
# fpr_train, tpr_train, _ = roc_curve(y_train, y_train_scores)
# fpr_test, tpr_test, _ = roc_curve(y_test, y_test_scores)
# auc_train = auc(fpr_train, tpr_train)
# auc_test = auc(fpr_test, tpr_test)
# print(auc_train)
# print(auc_test)

In [178]:
# import matplotlib.pyplot as plt
# plt.figure()
# plt.xlim([-0.01, 1.00])
# plt.ylim([-0.01, 1.01])
# plt.plot(fpr_train, tpr_train, lw=3, label='LogRegr ROC curve (area = {:0.2f})'.format(auc_train))
# plt.xlabel('False Positive Rate', fontsize=16)
# plt.ylabel('True Positive Rate', fontsize=16)
# plt.legend(loc='lower right', fontsize=13)
# plt.show()

In [179]:
# import matplotlib.pyplot as plt
# plt.figure()
# plt.xlim([-0.01, 1.00])
# plt.ylim([-0.01, 1.01])
# plt.plot(fpr_test, tpr_test, lw=3, label='LogRegr ROC curve (area = {:0.2f})'.format(auc_test))
# plt.xlabel('False Positive Rate', fontsize=16)
# plt.ylabel('True Positive Rate', fontsize=16)
# plt.legend(loc='lower right', fontsize=13)
# plt.show()

In [181]:
# bm = blight_model()
# res = 'Data type Test: '
# res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
# res += 'Data shape Test: '
# res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
# res += 'Data Values Test: '
# res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
# res += 'Data Values type Test: '
# res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
# res += 'Index type Test: '
# res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
# res += 'Index values type Test: '
# res += ['Failed: type(bm.index[0]) should be int64\n','Passed\n'][str(type(bm.index[0])).count("int64")>0]

# res += 'Output index shape test:'
# res += ['Failed, bm.index.shape should be (61001,)\n','Passed\n'][bm.index.shape==(61001,)]

# res += 'Output index test: '
# if bm.index.shape==(61001,):
#     res +=['Failed\n','Passed\n'][all(pd.read_csv('test.csv',usecols=[0],index_col=0).sort_index().index.values==bm.sort_index().index.values)]
# else:
#     res+='Failed'
# print(res)

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:33: FutureWarning: 
Setting NaNs in `categories` is deprecated and will be removed in a future version of pandas.
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:34: FutureWarning: 
Setting NaNs in `categories` is deprecated and will be removed in a future version of pandas.


Data type Test: Passed
Data shape Test: Passed
Data Values Test: Passed
Data Values type Test: Passed
Index type Test: Passed
Index values type Test: Passed
Output index shape test:Passed
Output index test: Passed



In [156]:
# bm.head()